# DRAS scenario generation


### library importation, constant definition and data loading from files

The next cell defines the imports nedeed to execute the notebook

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline
from math import sin, cos, sqrt, atan2, radians
pd.set_option('precision', 2)
from functools import reduce
import os
from operator import truediv
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import matplotlib.pyplot as plt
import numpy as np
import shutil
    


Populating the interactive namespace from numpy and matplotlib


The next cell defines the contanst used by the procedures bellow


In [2]:
instance_dir = '../dras-controller/instances/'

ins_file = 'ins'
ref_file = 'ref'
w_file = 'w'

schedule_duration  = 3600
K = 60
loss = [1,10,50]
p_loss=[0.6, 0.3, 0.1]
offset_loss = 30
base_task_mips = 60*1000
factor_task_mips = 3
deadline_factor = 30
server_mips = 1000


numpy.random.seed(seed=111)

INS   
1 5 // client-number server-number  
5 0.5 0.5 1 1 // client-server-number PAND PAD CRND CND  
800 20 1 // schedule-duration interval-number kw-dolar  

## ins

In [3]:
def generate_ins(example,CR):
    
    # create or overwrite folder
    my_dir = instance_dir + example
    if os.path.exists(my_dir):
        shutil.rmtree(my_dir)
    os.makedirs(my_dir)
    
    C = len(CR)
    R = sum(CR)
    file_name = instance_dir + example + '/'+  ins_file

    d = {'C': [C], 'R': [R]}
    first = pd.DataFrame(data=d)
    first_name = file_name + "_tmp1"
    first.to_csv(first_name, sep=' ', encoding='utf-8', index=False, header=False)


    d = {'RN': [i for i in CR], 
         'PAND': [0 for i in range(C)],
         'PAD': [0 for i in range(C)],
         'CRND': [0 for i in range(C)], 
         'CRD': [0 for i in range(C)]}

    second = pd.DataFrame(data=d)
    second_name = file_name + "_tmp2"
    second.to_csv(second_name, sep=' ', encoding='utf-8', index=False, header=False)

    d = {'schedule_duration': [schedule_duration],
         'K': [K],
         'kwdolar': [0]
        }
    third = pd.DataFrame(data=d)
    third_name = file_name + "_tmp3"
    third.to_csv(third_name, sep=' ', encoding='utf-8', index=False, header=False)

    filenames = [first_name,second_name,third_name]

    with open(file_name, 'w') as outfile:
        for fname in filenames:    
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)
            os.remove(fname)
        

## w



In [4]:
def generate_w(example,CR,tol,W): 
   
    C = len(CR)
    R = sum(CR)

    for clientId in range(C): 

        # read from worload exp1
        data = pd.DataFrame() 
        data['arriveTime'] = numpy.random.randint(0,3600, size=W[clientId])


        data = data.sort_values(by=['arriveTime'])
        data['type'] = numpy.random.randint(0,1 + 1, size=W[clientId] )
        data['mips'] = numpy.random.randint(base_task_mips,base_task_mips*factor_task_mips, size=W[clientId])


        data['deadline'] = data.apply(lambda e: random.randint(e.mips/server_mips * 20 ,
                                                               e.mips/server_mips * deadline_factor)
                                      ,axis = 1) 

        #data['loss'] = numpy.random.randint(base_loss,base_loss + offset_loss + 1, size=W)
        data['loss'] =  np.random.choice(loss, W[clientId], p=p_loss)
        data['tolerance'] =  np.random.choice([tol[clientId]], W[clientId], p=[1])


        file_name = instance_dir + example + '/'+  w_file
        file_name_client = file_name + str(clientId)    
        d = {'W': [W[clientId]]}
        first = pd.DataFrame(data=d)
        first_name = file_name_client + "_tmp1"
        first.to_csv(first_name, sep=' ', encoding='utf-8', index=False, header=False)
        d = {'clientId': [clientId for i in range(W[clientId])], 
             'mips': data['mips'],
             'arriveTime': data['arriveTime'],
             'deadline': data['deadline'], 
             'deferable': [1 for i in range(W[clientId])],
             'loss':  data['loss'],
             'type': data['type'],
             'tolerance': data['tolerance'] 
            }
        second = pd.DataFrame(data=d) 
        second_name = file_name_client + "_tmp2"
        second.to_csv(second_name, sep=' ', encoding='utf-8', index=False, header=False)

        filenames = [first_name,second_name]
        with open(file_name_client, 'w') as outfile:
            for fname in filenames:    
                with open(fname) as infile:
                    for line in infile:
                        outfile.write(line)
                    os.remove(fname)



## 9 escenarios homogeneos en tamaños y tipo de clientes



 DRD-5-0-0-hom-1 tol = [1.0,1.0,1.0,1.0,1.0]

DRD-5-0-0-hom-2 tol = [0.8,0.8,0.8,0.8,0.8]

%(chico, muy tolerante) x 5, DRD-5-0-0-hom-3 tol = [0.4,0.4,0.4,0.4,0.4]
%(mediano, poco tolerante) x 10, DRD-0-10-0-hom-1 tol = [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]
%(mediano, tolerancia media) x 10, DRD-0-10-0-hom-2 tol = [0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8]
%(mediano, muy tolerante) x 10, DRD-0-10-0-hom-3 tol = [0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,]
%(grandes, poco tolerante) x 30, DRD-0-0-30-hom-1 tol = [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]
%(grandes, tolerancia media) x 30, DRD-0-0-30-hom-2 tol = [0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8]
%(grandes, muy tolerante) x 30 DRD-0-0-30-hom-3 tol = [0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4]

In [5]:
# son los del trabajo anterior


## 9 escenarios homogeneos en tamaños y tolerancias
%DRD-5-0-0-het-1 tol = [1.0,1.0,1.0,0.8,0.4]

%DRD-5-0-0-het-2 tol = [1.0,8.0,0.8,0.4,0.4]

%DRD-5-0-0-het-3 tol = [1.0,1.0,0.8,0.8,0.4]

%DRD-0-10-0-het-1 tol = [1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4]

% DRD-0-10-0-het-2 tol = [1.0,8.0,0.8,0.4,0.4,1.0,8.0,0.8,0.4,0.4]

% DRD-0-10-0-het-3 tol = [1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4]

% DRD-0-0-30-het-1  tol = [1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4]

% DRD-0-0-30-het-2 tol = [1.0,8.0,0.8,0.4,0.4,1.0,8.0,0.8,0.4,0.4,1.0,8.0,0.8,0.4,0.4,1.0,8.0,0.8,0.4,0.4,1.0,8.0,0.8,0.4,0.4,1.0,8.0,0.8,0.4,0.4]

% DRD-0-0-30-het-3 tol = [1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4]

In [6]:
name = 'DRD-5-0-0-het-1'
# vector de clientes x servers
CR = [5,5,5,5,5]
W = [1500,1500,1500,1500,1500]
tol = [1.0,1.0,1.0,0.8,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-5-0-0-het-2'
# vector de clientes x servers
CR = [5,5,5,5,5]
W = [1500,1500,1500,1500,1500]
tol = [1.0,0.8,0.8,0.4,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-5-0-0-het-3'
# vector de clientes x servers
CR = [5,5,5,5,5]
W = [1500,1500,1500,1500,1500]
tol = [1.0,1.0,0.8,0.8,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-0-10-0-het-1'
# vector de clientes x servers
CR = [15,15,15,15,15,15,15,15,15,15]
W = [6000,6000,6000,6000,6000,6000,6000,6000,6000,6000]
tol = [1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-0-10-0-het-2'
# vector de clientes x servers
CR = [15,15,15,15,15,15,15,15,15,15]
W = [6000,6000,6000,6000,6000,6000,6000,6000,6000,6000]
tol = [1.0,0.8,0.8,0.4,0.4,1.0,0.8,0.8,0.4,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-0-10-0-het-3'
# vector de clientes x servers
CR = [15,15,15,15,15,15,15,15,15,15]
W = [6000,6000,6000,6000,6000,6000,6000,6000,6000,6000]
tol = [1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-0-0-30-het-1'
# vector de clientes x servers
CR = [30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30]
W = [9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,
     9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000]
tol = [1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4,1.0,
       1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-0-0-30-het-2'
# vector de clientes x servers
CR = [30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30]
W = [9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,
     9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000]
tol = [1.0,0.8,0.8,0.4,0.4,1.0,0.8,0.8,0.4,0.4,1.0,0.8,0.8,0.4,0.4,1.0,
       0.8,0.8,0.4,0.4,1.0,0.8,0.8,0.4,0.4,1.0,0.8,0.8,0.4,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-0-0-30-het-3'
# vector de clientes x servers
CR = [30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30]
W = [9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,
     9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000]
tol = [1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4,1.0,
       1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)


%\textbf{9 escenarios heterogeneos en tamaños, heterogeneos en tolerancias}\\

%DRD-3-1-1-het-1 tol = [1.0,1.0,1.0,0.8,0.4]

%DRD-1-1-3-het-2 tol = [1.0,8.0,0.8,0.4,0.4]

%DRD-1-3-1-het-3 tol = [1.0,1.0,0.8,0.8,0.4]

%DRD-6-2-2-het-1 tol = [1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4]

% DRD-2-6-2-het-2 tol = [1.0,8.0,0.8,0.4,0.4,1.0,8.0,0.8,0.4,0.4]

% DRD-2-2-6-het-3 tol = [1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4]

% DRD-20-5-5-het-1  tol = [1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4]

% DRD-5-20-5-het-2 tol = [1.0,8.0,0.8,0.4,0.4,1.0,8.0,0.8,0.4,0.4,1.0,8.0,0.8,0.4,0.4,1.0,8.0,0.8,0.4,0.4,1.0,8.0,0.8,0.4,0.4,1.0,8.0,0.8,0.4,0.4]

% DRD-5-5-20-het-3 tol = [1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4]

In [7]:
name = 'DRD-3-1-1-het-1'
# vector de clientes x servers
CR = [5,5,5,15,30]
W = [1500,1500,1500,6000,9000]
tol = [1.0,1.0,1.0,0.8,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-1-1-3-het-1'
# vector de clientes x servers
CR = [5,15,30,30,30]
W = [1500,6000,9000,9000,9000]
tol = [1.0,0.8,0.8,0.4,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-1-3-1-het-1'
# vector de clientes x servers
CR = [5,15,15,15,30]
W = [1500,6000,6000,6000,9000]
tol = [1.0,1.0,0.8,0.8,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-6-2-2-het-1'
# vector de clientes x servers
CR = [5,5,5,5,5,5,15,15,30,30]
W = [1500,1500,1500,1500,1500,1500,6000,6000,9000,9000]
tol = [1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-2-6-2-het-1'
# vector de clientes x servers
CR = [5,5,15,15,15,15,15,15,30,30]
W = [1500,1500,6000,6000,6000,6000,6000,6000,9000,9000]
tol = [1.0,0.8,0.8,0.4,0.4,1.0,0.8,0.8,0.4,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-2-2-6-het-1'
# vector de clientes x servers
CR = [5,5,15,15,30,30,30,30,30,30]
W = [1500,1500,6000,6000,9000,9000,9000,9000,9000,9000]
tol = [1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-20-5-5-het-1'
# vector de clientes x servers
CR = [5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,15,15,15,15,15,30,30,30,30,30]
W = [1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,
     1500,1500,6000,6000,6000,6000,6000,9000,9000,9000,9000,9000]
tol = [1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4,1.0,
       1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4,1.0,1.0,1.0,0.8,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-5-20-5-het-1'
# vector de clientes x servers
CR = [5,5,5,5,5,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,30,30,30,30,30]
W = [1500,1500,1500,1500,1500,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,
     6000,6000,6000,6000,6000,6000,6000,9000,9000,9000,9000,9000]
tol = [1.0,0.8,0.8,0.4,0.4,1.0,0.8,0.8,0.4,0.4,1.0,0.8,0.8,0.4,0.4,1.0,
       0.8,0.8,0.4,0.4,1.0,0.8,0.8,0.4,0.4,1.0,0.8,0.8,0.4,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-5-5-20-het-1'
# vector de clientes x servers
CR = [5,5,5,5,5,15,15,15,15,15,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30]
W = [1500,1500,1500,1500,1500,6000,6000,6000,6000,6000,9000,9000,9000,9000,9000,9000,9000,9000,
     9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000]
tol = [1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4,1.0,
       1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4,1.0,1.0,0.8,0.8,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

In [8]:
name = 'DRD-3-1-1-hom-1'
# vector de clientes x servers
CR = [5,5,5,15,30]
W = [1500,1500,1500,6000,9000]
tol = [1.0,1.0,1.0,1.0,1.0]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-1-1-3-hom-1'
# vector de clientes x servers
CR = [5,15,30,30,30]
W = [1500,6000,9000,9000,9000]
tol = [0.8,0.8,0.8,0.8,0.8]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-1-3-1-hom-1'
# vector de clientes x servers
CR = [5,15,15,15,30]
W = [1500,6000,6000,6000,9000]
tol = [0.4,0.4,0.4,0.4,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-6-2-2-hom-1'
# vector de clientes x servers
CR = [5,5,5,5,5,5,15,15,30,30]
W = [1500,1500,1500,1500,1500,1500,6000,6000,9000,9000]
tol = [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-2-6-2-hom-1'
# vector de clientes x servers
CR = [5,5,15,15,15,15,15,15,30,30]
W = [1500,1500,6000,6000,6000,6000,6000,6000,9000,9000]
tol = [0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-2-2-6-hom-1'
# vector de clientes x servers
CR = [5,5,15,15,30,30,30,30,30,30]
W = [1500,1500,6000,6000,9000,9000,9000,9000,9000,9000]
tol = [0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-20-5-5-hom-1'
# vector de clientes x servers
CR = [5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,15,15,15,15,15,30,30,30,30,30]
W = [1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,
     1500,1500,6000,6000,6000,6000,6000,9000,9000,9000,9000,9000]
tol = [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,
       1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-5-20-5-hom-1'
# vector de clientes x servers
CR = [5,5,5,5,5,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,30,30,30,30,30]
W = [1500,1500,1500,1500,1500,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,
     6000,6000,6000,6000,6000,6000,6000,9000,9000,9000,9000,9000]
tol = [0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.4,0.8,
       0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)

name = 'DRD-5-5-20-hom-1'
# vector de clientes x servers
CR = [5,5,5,5,5,15,15,15,15,15,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30]
W = [1500,1500,1500,1500,1500,6000,6000,6000,6000,6000,9000,9000,9000,9000,9000,9000,9000,9000,
     9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000]
tol = [0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,
       0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4]
generate_ins(name,CR)  
generate_w(name,CR,tol,W)